In [165]:
import bs4 as bs
import datetime as dt
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import csv
import json
from bson.objectid import ObjectId
import uuid
import re

In [177]:
shop_collection, drink_collection, review_collection = list(), list(), list()

In [178]:
def extractReviews(drinkId):
    review_ids = list()
    avgRating = 0
    numberOfReviews = len(driver.find_elements_by_xpath('//div[@class=" margin-b4__09f24__jfnOz border-color--default__09f24__NPAKY"]/div[1]/div[1]/div[2]/div[1]/span'))
    for index in range(numberOfReviews):
        review_object = dict()
        review = driver.find_elements_by_xpath('//div[@class=" margin-b4__09f24__jfnOz border-color--default__09f24__NPAKY"]')[index]
        print("Review: " + str(index))
        # Reviewer Name
        try:
            review_object["reviewerName"] = str(review.find_element_by_xpath('./div[1]/div[1]/div[2]/div[1]/span').text)
        except:
            review_object["reviewerName"] = ""
        # Review Rating
        try:
            reviewRating = str(review.find_element_by_xpath('./div[2]/span/div').get_attribute("aria-label"))
            reviewRating = float(re.findall(r"[-+]?(?:\d*\.\d+|\d+)", reviewRating)[0])
            review_object["reviewRating"] = reviewRating
        except:
            review_object["reviewRating"] = None
        # Review Date
        try:
            review_object["reviewDate"] = str(review.find_element_by_xpath('./div[2]/span[2]').text)
        except:
            review_object["reviewDate"] = ""
        # Review Message
        try:
            review.find_element_by_xpath('./div[3]/button').click()
            review_object["reviewMessage"] = str(review.find_element_by_xpath('./div[3]/div/p/span').text)
        except:
            review_object["reviewMessage"] = ""
        
        reviewId = ObjectId(uuid.uuid1().hex[:24])
        review_object["_id"] = reviewId
        review_object["drinkId"] = drinkId
        
        print(review_object)
        avgRating += reviewRating
        review_collection.append(review_object)
        review_ids.append(reviewId)
    
    if review_ids:
        avgRating = avgRating / len(review_ids)
    return review_ids, avgRating

In [179]:
def visitShops(shopId, shopName):
    # driver = webdriver.Chrome(executable_path="./chromedriver.exe")
    # driver.implicitly_wait(0.5)
    # driver.get("https://www.yelp.ca/biz/tika-tea-house-toronto-2?osq=boba+shops")
    # shopId = ObjectId(uuid.uuid1().hex[:24])

    drink_ids = []
    # Open Each Drink.
    numberOfDrinks = len(driver.find_elements_by_xpath("//a[@class='css-1fliqpp']/div/div/div"))
    for index in range(numberOfDrinks):
        drink_object = dict()
        drink_modal = driver.find_elements_by_xpath("//a[@class='css-1fliqpp']")[index]
        drink_modal.click()
        # Get the Drink.
        drinkId = ObjectId(uuid.uuid1().hex[:24])
        drink_object["_id"] = drinkId

        # Extract Drink Name
        drinkName = "Unknown Drink"
        try:
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//h2[@class="css-h19vjk"]')))
            drinkName = driver.find_element_by_xpath('//h2[@class="css-h19vjk"]').text
        except:
            print("Couldn't Find Drink Name!")
            drink_modal.send_keys(Keys.ESCAPE)
            continue
        print(drinkName)
        drink_object["drinkName"] = drinkName

        # Extract Drink Image
        try:
            drink_object["drinkImage"] = str(driver.find_element_by_xpath('//*[@id="main-content"]/section[1]/div[3]/div[2]/div/div/div[1]/div[1]/img').get_attribute("src"))
        except:
            drink_object["drinkImage"] = ""

        # Open Review Section
        try:
            driver.find_element_by_xpath("//*[@id='main-content']/section[1]/div[3]/div[2]/div/div/div[2]/div/div[2]/div/div/div[2]/button").click()
        except:
            print("Drink modal doesn't exist!")
            driver.find_element_by_xpath('//*[@id="main-content"]/section[1]/div[3]/div[2]/div/div/div[2]/div/div[1]/div/div/div/div[2]/button/span/span').click()
        
        # Extract Reviews
        review_ids, avgRating = extractReviews(drinkId)
        drink_object["avgRating"] = avgRating
        drink_object["reviews"] = review_ids
        drink_object["shopId"] = shopId
        drink_object["shopName"] = shopName
        
        if review_ids:
            drink_collection.append(drink_object)
            drink_ids.append(drinkId)

        # Close Modal
        driver.find_element_by_xpath('//*[@id="main-content"]/section[1]/div[3]/div[2]/div/div/div[2]/div/div[1]/div/div/div/div[2]/button/span/span').click()

    return drink_ids

In [ ]:
driver = webdriver.Chrome(executable_path="./chromedriver.exe")
driver.implicitly_wait(0.5)

# Access Yelp's Top 10 Boba Spots.
driver.get("https://www.yelp.ca/search?find_desc=boba+shops&find_loc=Toronto%2C+ON&sortby=review_count")

# Extract the Top 10 Boba shop links.
# If no sponsered shops:
# shop_list = driver.find_elements_by_xpath('//*[@id="main-content"]/div/ul/li/div/div/div/div[2]/div[1]/div[1]/div[1]/div/div/h3/span/a')
# start_index = 4
# If sponsered shops exist:
shops_list = driver.find_elements_by_xpath('//*[@id="main-content"]/div/ul/li[9]//following::node()/a[@class="css-1m051bw"]')
start_index = 9

for index in range(len(shops_list)-1):
    shop_object = dict()
    shop = driver.find_element_by_xpath('//*[@id="main-content"]/div/ul/li[{}]//following::node()/a[@class="css-1m051bw"]'.format(index+start_index))

    # Generate random shop id.
    shopId = ObjectId(uuid.uuid1().hex[:24])
    shop_object["_id"] = shopId
    
    # Get shop name.
    shopName = str(shop.get_attribute('name'))
    shop_object["shopName"] = shopName
    print("*********|| " + shopName + " ||*********")

    # Get shop image.
    shopImage = str(driver.find_element_by_xpath('//*[@id="main-content"]/div/ul/li[{}]/div/div/div/div[1]/div/div/div/div/div/div[1]/div/a/img'.format(index+start_index)).get_attribute("src"))
    shop_object["shopImage"] = shopImage

    # Get shop rating.
    avgRating = str(driver.find_element_by_xpath('//*[@id="main-content"]/div/ul/li[{}]/div/div/div/div[2]/div[1]/div[1]/div[2]/div/div/div[1]/span/div'.format(index+start_index)).get_attribute("aria-label"))
    avgRating = float(re.findall(r"[-+]?(?:\d*\.\d+|\d+)", avgRating)[0])
    shop_object["avgRating"] = avgRating

    # Open and Switch Tabs
    shop.send_keys(Keys.CONTROL + Keys.RETURN)

    # Switch Tabs.
    try:
        parent = driver.window_handles[0]
        child = driver.window_handles[1]
        driver.switch_to.window(child)
    except:
        print()

    # Get shop address.
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, ' //p[@class=" css-qyp8bo"]')))
    shopAddress = driver.find_element_by_xpath(' //p[@class=" css-qyp8bo"]').text
    shop_object["shopAddress"] = shopAddress

    drink_ids = visitShops(shopId, shopName)
    shop_object["drinks"] = drink_ids
    if drink_ids:
        shop_collection.append(shop_object)

    # Close tab and switch focus tabs
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [172]:
driver.quit()

In [182]:
# Export review_collection as json.
reviews_df = pd.DataFrame(review_collection)
reviews_df["_id"] = reviews_df["_id"].apply(lambda x: {"$oid": str(x)})
reviews_df["drinkId"] = reviews_df["drinkId"].apply(lambda x: {"$oid": str(x)})
reviews_df.to_json("../data/boba_reviews.json", orient='records')

In [183]:
# Export drink_collection as json.
drinks_df = pd.DataFrame(drink_collection)
drinks_df["_id"] = drinks_df["_id"].apply(lambda x: {"$oid": str(x)})
drinks_df["shopId"] = drinks_df["shopId"].apply(lambda x: {"$oid": str(x)})
drinks_df["reviews"] = drinks_df["reviews"].apply(lambda x: [{"$oid": str(id)} for id in x])
drinks_df.to_json("../data/boba_drinks.json", orient='records')

In [184]:
# Export shop_collection as json.
shop_df = pd.DataFrame(shop_collection)
shop_df["_id"] = shop_df["_id"].apply(lambda x: {"$oid": str(x)})
shop_df["drinks"] = shop_df["drinks"].apply(lambda x: [{"$oid": str(id)} for id in x])
shop_df.to_json("../data/boba_shops.json", orient='records')

In [203]:
drink_collection_df = pd.DataFrame(drink_collection)
drink_collection_df["reviewLength"] = drink_collection_df["reviews"].apply(lambda x: len(x))
drink_collection_df.sort_values(by="reviewLength", ascending=False).head(15)

,_id,drinkName,drinkImage,avgRating,reviews,shopId,shopName,reviewLength
0,b50ef315009611ed9159d0ab,Roasted Milk Tea,https://s3-media0.fl.yelpcdn.com/bphoto/HrWnzP...,3.600000,"[b5e3d071009611eda2b8d0ab, b62c97e1009611ed8cf...",b07d5cb4009611edbd66d0ab,Chatime - Dundas,15
27,f2535988009611ed87d2d0ab,Matcha Latte,https://s3-media0.fl.yelpcdn.com/bphoto/OxEyuB...,4.400000,"[f32fde8a009611edb2e9d0ab, f37adffc009611ed808...",eed07be4009611eda7a7d0ab,Icha Tea,15
28,f63cd502009611ed91acd0ab,Rose Jasmine Tea,https://s3-media0.fl.yelpcdn.com/bphoto/T9nrg2...,3.733333,"[f740fe55009611ed8b54d0ab, f7c49849009611ed8cc...",eed07be4009611eda7a7d0ab,Icha Tea,15
12,d4614d97009611ed8ed2d0ab,Alley Trio Milk Tea,https://s3-media0.fl.yelpcdn.com/bphoto/hDLrfK...,4.466667,"[d5124dca009611edad62d0ab, d54cbec4009611eda04...",c75c2ba7009611ed8cd7d0ab,The Alley,15
11,cfe4765f009611edaf23d0ab,Royal No.9 Milk Tea,https://s3-media0.fl.yelpcdn.com/bphoto/1n6DRf...,4.466667,"[d07a633f009611eda94bd0ab, d0af2efa009611edb86...",c75c2ba7009611ed8cd7d0ab,The Alley,15
10,cb5bacc9009611ed9f07d0ab,Iron Goddess Milk Tea,https://s3-media0.fl.yelpcdn.com/bphoto/LSZdkk...,4.400000,"[cc3d5a08009611edac37d0ab, cc9d2d73009611ed934...",c75c2ba7009611ed8cd7d0ab,The Alley,15
1,b9c251dd009611eda86dd0ab,Brown Sugar Pearl Milk Tea,https://s3-media0.fl.yelpcdn.com/bphoto/dYm7xx...,4.153846,"[ba600469009611edad9ad0ab, ba8b238b009611edad5...",b07d5cb4009611edbd66d0ab,Chatime - Dundas,13
13,d80b4384009611edba0ad0ab,Brown Sugar Deerioca,https://s3-media0.fl.yelpcdn.com/bphoto/0_g60W...,4.307692,"[d89431e1009611edaf28d0ab, d8c505d6009611edb56...",c75c2ba7009611ed8cd7d0ab,The Alley,13
14,daeeed60009611eda901d0ab,Deerioca Fever,https://s3-media0.fl.yelpcdn.com/bphoto/VR8ajd...,4.545455,"[db84b92e009611ed942ed0ab, dbb78904009611eda17...",c75c2ba7009611ed8cd7d0ab,The Alley,11
30,fd574866009611edbb8fd0ab,Classic Milk Tea,https://s3-media0.fl.yelpcdn.com/bphoto/JM3Tvo...,4.000000,"[fe409bdc009611ed9c87d0ab, fe979545009611eda94...",eed07be4009611eda7a7d0ab,Icha Tea,10
